<a href="https://colab.research.google.com/github/lcbjrrr/quant/blob/master/J_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Topic:** AI/ML

**Title:** Classifiers (Decision Tree)

**Author:** Luiz Barboza

**Date:** 20/dec/22

**Lang:** Julia

**Site:** https://quant-research.group/

**Email:** contato@quant-research.group


# Julia Installation

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.3 on the current Colab Runtime...
2023-01-05 12:39:59 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.3-linux-x86_64.tar.gz [130030846/130030846] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [1]:
versioninfo()

Julia Version 1.8.3
Commit 0434deb161e (2022-11-14 20:14 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  LD_PRELOAD = /usr/lib/x86_64-linux-gnu/libtcmalloc.so.4
  JULIA_NUM_THREADS = 2


# Classifiers

In [2]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Lathe") 
Pkg.add("DecisionTree") 
Pkg.add("StatsBase") 
Pkg.add("EvalMetrics") 

using CSV
using DataFrames
using Lathe
using Lathe.preprocess: TrainTestSplit
using DecisionTree
using StatsBase
using EvalMetrics

  ✓ EvalMetrics
  ✓ SpecialFunctions
  ✓ StatsFuns
  ✓ Distributions
  ✓ Lathe
  9 dependencies successfully precompiled in 23 seconds. 62 already precompiled.


In [3]:
;wget https://raw.githubusercontent.com/lcbjrrr/data/main/gender%20-%20all.csv

--2023-01-05 12:45:00--  https://raw.githubusercontent.com/lcbjrrr/data/main/gender%20-%20all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267751 (261K) [text/plain]
Saving to: ‘gender - all.csv’

     0K .......... .......... .......... .......... .......... 19% 4.16M 0s
    50K .......... .......... .......... .......... .......... 38% 5.56M 0s
   100K .......... .......... .......... .......... .......... 57% 26.5M 0s
   150K .......... .......... .......... .......... .......... 76% 17.6M 0s
   200K .......... .......... .......... .......... .......... 95% 7.87M 0s
   250K .......... .                                          100%  210M=0.03s

2023-01-05 12:45:00 (8.13 MB/s) - ‘gender - all.csv’ saved [267751/267751]



In [4]:
#read csv
df=CSV.read("gender - all.csv", DataFrame)
first(df,5)

,G,H,W
,Int64,Float64,Float64
1,0,187.571,109.952
2,0,174.706,73.7775
3,0,188.24,96.7004
4,0,182.197,100.019
5,0,177.5,93.7954


## Decision Tree

In [5]:
train, test = TrainTestSplit(df,.80)

(8019×3 DataFrame
  Row │ G      H        W        
      │ Int64  Float64  Float64  
──────┼──────────────────────────
    1 │     0  187.571  109.952
    2 │     0  174.706   73.7775
    3 │     0  188.24    96.7004
    4 │     0  182.197  100.019
    5 │     0  177.5     93.7954
    6 │     0  170.823   69.1873
    7 │     0  173.605   76.3505
    8 │     0  161.179   71.0908
    9 │     0  180.836   84.8204
   10 │     0  181.968   97.1551
   11 │     0  175.979   86.1119
  ⋮   │   ⋮       ⋮        ⋮
 8010 │     1  154.826   43.4939
 8011 │     1  159.097   46.7516
 8012 │     1  157.678   63.0417
 8013 │     1  152.477   44.4034
 8014 │     1  150.11    50.2408
 8015 │     1  168.079   62.1716
 8016 │     1  170.351   77.6672
 8017 │     1  162.225   58.3979
 8018 │     1  175.347   74.4784
 8019 │     1  157.338   51.6587
                7998 rows omitted, 1981×3 DataFrame
  Row │ G      H        W        
      │ Int64  Float64  Float64  
──────┼──────────────────────────
    1 

In [6]:
tree = DecisionTreeClassifier(max_depth=3)
DecisionTree.fit!(tree, Matrix(train[:,[:H,:W]]), train[:,:G])     

DecisionTreeClassifier
max_depth:                3
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [0, 1]
root:                     Decision Tree
Leaves: 8
Depth:  3

In [7]:
function accuracy(y,pred)
  acc = sum(pred.==y)/length(y)
  return acc
end

accuracy (generic function with 1 method)

In [8]:
pred_train = DecisionTree.predict(tree,Matrix(train[:,[:H,:W]]))
acc_train = accuracy(train.G,pred_train)
print("Accuracy (Train): ",acc_train)

Accuracy (Train): 0.9072203516647961

In [9]:
pred_test=DecisionTree.predict(tree,Matrix(test[:,[:H,:W]]))
acc_test = accuracy(test.G,pred_test)
print("Accuracy (Test): ",acc_test)

Accuracy (Test): 0.9126703685007572

# Classification Metrics

In [11]:
#Confusion Matrix
counts(test.G,pred_test)

2×2 Matrix{Int64}:
 880   95
  78  928

In [16]:
# Precision/Recall
println("Precision: ",precision(test.G,pred_test))
println("Recall: " , recall(test.G,pred_test))

Precision: 0.9071358748778103
Recall: 0.9224652087475149
